# Capstone Workbook 6: Advanced Modelling

This section will build upon the initial modelling in chapter 4. For this advanced modelling, the embedded text data will be incorporated into the models, significiantly increasing the quantity of data that is being processed.

Lasso and XGBoost Regressor models will be used. These models will be subject to a range of hyperparameter tuning, being used across multiple iterations to model different forms of the independent dataframes.

Model success will be evaluated and compared, with feature analysis being conducted through model outputs including coefficients, feature importance and Shapley values. 

In [1]:
# Import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import statsmodels.api as sm
import sklearn

In [2]:
# importing the required packages 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
# To do a cross-validated grid search
from sklearn.model_selection import GridSearchCV

In [3]:
# Additional required imports 

# To "pickle" things, like accuracies or even an entire fitted model
import joblib

# To set up a temporary directory for caching pipeline results
from tempfile import mkdtemp

from sklearn.metrics import mean_absolute_error, r2_score

# Completing a Model With the Processed Text Columns

The two dataframes created during workbook 5, will be imported. These two dataframes consist of the dataframes with the text processed information, where the listing title on one dataframe has been count vectorized and on the other it has been TF-IDF.

In [4]:
# import the listing title count vectorized column
airbnb_ldn_cv = pd.read_csv('data/airbnb_ldn_cv.csv')

# import the listing title tf-idf column:
airbnb_ldn_tfidf = pd.read_csv('data/airbnb_ldn_tfidf.csv')

The independent variables will be defined for both dataframes, as well as the target column: 

In [5]:
# on the cv dataframe:
X_cv = airbnb_ldn_cv.drop(columns = 'Annual Revenue LTM (Native)')

# on the tf-idf dataframe:
X_tfidf = airbnb_ldn_tfidf.drop(columns = 'Annual Revenue LTM (Native)')

# define the target column:
y = airbnb_ldn_cv['Annual Revenue LTM (Native)']

Columns previously deemed semantically irrelevant for the purpose of these models will be dropped from both dataframes (as they were in the initial modelling stage):

In [6]:
X_cv.drop(columns = ['Latitude', 'Longitude', 'Airbnb Host ID', 'Number of Bookings LTM - Number of observed month'], inplace=True)
X_tfidf.drop(columns = ['Latitude', 'Longitude', 'Airbnb Host ID', 'Number of Bookings LTM - Number of observed month'], inplace=True)

Before additional modelling can take place, the continuous variables need to be scaled. The continuous variables were previously identified and have not changed. The continuous columns will be scaled for both the X_cv and X_tfidf dataframes

The previously identified continuous columns are:

- 'Number of Reviews', 
- 'Bedrooms', 
- 'Bathrooms', 
- 'Max Guests',
- 'Cleaning Fee (Native)', 
- 'Extra People Fee(Native)', 
- 'Minimum Stay',
- 'Overall Rating', 
- 'Airbnb Communication Rating',
- 'Airbnb Accuracy Rating', 
- 'Airbnb Checkin Rating',
- 'Airbnb Location Rating', 
- 'Airbnb Value Rating',
- 'Count Available Days LTM', 
- 'Count Blocked Days LTM',
- 'Count Reservation Days LTM', 
- 'Occupancy Rate LTM',
- 'Number of Bookings LTM', 
- 'Average Daily Rate (Native)'

These will be isolated into a new dataframe - using the X_tfidf dataframe as the initial dataframe for the columns to be selected from:

In [7]:
X_cont = X_tfidf[['Number of Reviews', 'Bedrooms', 'Bathrooms', 'Max Guests',
               'Cleaning Fee (Native)', 'Extra People Fee(Native)', 'Minimum Stay',
               'Overall Rating', 'Airbnb Communication Rating',
               'Airbnb Accuracy Rating', 'Airbnb Checkin Rating',
               'Airbnb Location Rating', 'Airbnb Value Rating',
               'Count Available Days LTM', 'Count Blocked Days LTM',
               'Count Reservation Days LTM', 'Occupancy Rate LTM',
               'Number of Bookings LTM', 'Average Daily Rate (Native)']]

Now a specific dataframe has been created for the continuous columns. These columns will be scaled using 'StandardScaler()':

In [8]:
# import the required scaler
from sklearn.preprocessing import StandardScaler

In [9]:
# scale the continuous columns:
X_cont_s = StandardScaler().fit_transform(X_cont)

# convert the scaled values back into a dataframe:
X_cont_s = pd.DataFrame(X_cont_s, columns = X_cont.columns)

In [10]:
X_cont_s

,Number of Reviews,Bedrooms,Bathrooms,Max Guests,Cleaning Fee (Native),Extra People Fee(Native),Minimum Stay,Overall Rating,Airbnb Communication Rating,Airbnb Accuracy Rating,Airbnb Checkin Rating,Airbnb Location Rating,Airbnb Value Rating,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Average Daily Rate (Native)
0,-0.402287,0.567055,0.832637,1.329516,3.085459,-0.321408,-0.105203,-1.048155,-0.632615,-1.367424,-0.616331,-1.840114,-1.029263,-1.122503,1.122503,-0.999760,-1.246128,-0.631034,0.490704
1,-0.368282,0.567055,-0.589729,0.340926,-0.585405,-0.321408,-0.220206,0.323599,0.433841,-0.399818,0.462298,-0.642164,0.780512,0.409072,-0.409072,0.181568,-0.193335,0.381395,-0.113132
2,-0.538303,-0.535012,0.832637,-0.153368,-0.585405,-0.321408,0.124801,0.715529,0.433841,-1.367424,0.462298,-1.840114,0.780512,-0.481844,0.481844,-1.054706,-2.727837,-0.671531,0.417552
3,-0.215264,0.567055,-0.589729,0.340926,0.239828,-0.014539,0.009799,0.323599,0.433841,0.567788,0.462298,-0.642164,-0.124376,0.158815,-0.158815,0.415086,0.859458,-0.145068,-0.502168
4,-0.555305,-0.535012,-0.589729,-0.647663,-0.585405,-0.321408,0.009799,-1.244119,0.433841,0.567788,0.462298,-0.642164,-0.124376,-1.222606,1.222606,-1.068442,-0.661243,-0.671531,-0.838667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32669,-0.521301,-0.535012,0.832637,-0.647663,0.052491,-0.321408,-0.105203,-1.244119,0.433841,-2.335031,-3.852220,-4.236012,-2.839037,-0.181535,0.181535,-0.505251,-0.973182,-0.550039,-0.413055
32670,-0.351280,-0.535012,-0.589729,-0.647663,-0.210730,-0.321408,-0.220206,0.323599,0.433841,0.567788,0.462298,-0.642164,-0.124376,-0.842215,0.842215,-1.040969,-2.571868,-0.509542,-0.764850
32671,-0.368282,-0.535012,-0.589729,-0.647663,-0.585405,-0.321408,-0.162704,0.323599,0.433841,0.567788,-0.616331,-0.642164,0.780512,1.089773,-1.089773,0.401350,-0.622251,0.016921,-0.806746
32672,-0.453293,-0.535012,-0.589729,-0.647663,-0.341155,0.930618,-0.105203,0.715529,0.433841,0.567788,0.462298,-0.642164,-0.124376,0.078733,-0.078733,-0.367887,-0.973182,-0.226062,-0.681057


Use the scaled continuous values to replace the unscaled continuous values on the original X_cv and X_tfidf dataframes:

In [11]:
# scale the X_cv column:
X_cv[['Number of Reviews', 'Bedrooms', 'Bathrooms', 'Max Guests',
       'Cleaning Fee (Native)', 'Extra People Fee(Native)', 'Minimum Stay',
       'Overall Rating', 'Airbnb Communication Rating',
       'Airbnb Accuracy Rating', 'Airbnb Checkin Rating',
       'Airbnb Location Rating', 'Airbnb Value Rating',
       'Count Available Days LTM', 'Count Blocked Days LTM',
       'Count Reservation Days LTM', 'Occupancy Rate LTM',
       'Number of Bookings LTM', 'Average Daily Rate (Native)']] = X_cont_s[['Number of Reviews', 'Bedrooms', 'Bathrooms', 'Max Guests',
                                                                               'Cleaning Fee (Native)', 'Extra People Fee(Native)', 'Minimum Stay',
                                                                               'Overall Rating', 'Airbnb Communication Rating',
                                                                               'Airbnb Accuracy Rating', 'Airbnb Checkin Rating',
                                                                               'Airbnb Location Rating', 'Airbnb Value Rating',
                                                                               'Count Available Days LTM', 'Count Blocked Days LTM',
                                                                               'Count Reservation Days LTM', 'Occupancy Rate LTM',
                                                                               'Number of Bookings LTM', 'Average Daily Rate (Native)']]

In [12]:
# separate dataframe to scale X_tfidf - the unscaled version will be used in later modelling:
X_tfidf_scaled = X_tfidf

In [13]:
# scale the X_tfidf column:
X_tfidf_scaled[['Number of Reviews', 'Bedrooms', 'Bathrooms', 'Max Guests',
       'Cleaning Fee (Native)', 'Extra People Fee(Native)', 'Minimum Stay',
       'Overall Rating', 'Airbnb Communication Rating',
       'Airbnb Accuracy Rating', 'Airbnb Checkin Rating',
       'Airbnb Location Rating', 'Airbnb Value Rating',
       'Count Available Days LTM', 'Count Blocked Days LTM',
       'Count Reservation Days LTM', 'Occupancy Rate LTM',
       'Number of Bookings LTM', 'Average Daily Rate (Native)']] = X_cont_s[['Number of Reviews', 'Bedrooms', 'Bathrooms', 'Max Guests',
                                                                               'Cleaning Fee (Native)', 'Extra People Fee(Native)', 'Minimum Stay',
                                                                               'Overall Rating', 'Airbnb Communication Rating',
                                                                               'Airbnb Accuracy Rating', 'Airbnb Checkin Rating',
                                                                               'Airbnb Location Rating', 'Airbnb Value Rating',
                                                                               'Count Available Days LTM', 'Count Blocked Days LTM',
                                                                               'Count Reservation Days LTM', 'Occupancy Rate LTM',
                                                                               'Number of Bookings LTM', 'Average Daily Rate (Native)']]

## Model 1: Lasso Model
### M1 - V1: Count Vectorized Listing Title DataFrame

Starting with the count vectorized dataframe (X_cv), a train test split will be complete and the training data will be passed through the Lasso regression model:

In [14]:
# required imports for initial linear regression model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso

In [15]:
# split independent and target data between train and test sets:
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_cv, y, test_size = 0.25, random_state=42)

In [16]:
# 1. Instantiate the model
alpha = 0.1
max_iter = 10000
lasso_cv = Lasso(alpha = alpha, max_iter = max_iter)

# 2. Fit the model
lasso_cv.fit(X_train_cv, y_train_cv)

# 3. Determine the model coefficients
lasso_coef = lasso_cv.coef_
feature_names = X_train_cv.columns

coef_cv = pd.DataFrame({'Features': feature_names, 'Coefficients': lasso_coef})

In [17]:
# Make predictions
y_pred_train_cv = lasso_cv.predict(X_train_cv)
y_pred_cv = lasso_cv.predict(X_test_cv)



# Evaluate model
print(f'Train MAE --- {round(mean_absolute_error(y_train_cv, y_pred_train_cv),1)}')
print(f'Test MAE ---- {round(mean_absolute_error(y_test_cv, y_pred_cv),1)}')
print(f'Train R2 ---- {round(r2_score(y_train_cv, y_pred_train_cv),3)}')
print(f'Test R2 ----- {round(r2_score(y_test_cv, y_pred_cv),3)}')

Train MAE --- 6573.9
Test MAE ---- 6761.1
Train R2 ---- 0.725
Test R2 ----- 0.719


This first model has performed very well. The accuracy score for the training and test datasets are both a very similar score (less than 0.01 difference) and both above 0.7. The train and test MAE are then both very similar and not too large when considering the overall scale of the range in total revenue. 

The coefficients will now be evaluated further:

In [18]:
# view the top 25 values:
(coef_cv.sort_values(by='Coefficients', ascending=False)).head(25)

,Features,Coefficients
342,wc2a,33202.588802
85,Stair gates,20876.297144
547,covent,20745.425220
202,ec3a,19529.817865
291,sw1a,17301.786428
858,oxford circus,16895.417279
757,london eye,14183.499759
860,oxford street,11962.500969
347,wc2r,11576.145782
199,ec2r,11257.047749


In [19]:
# view columns will a coefficient of 0:
print(f"There are {len(coef_cv[coef_cv['Coefficients']==0])} columns with a coefficient of zero.")

print("These columns are:")
coef_cv[coef_cv['Coefficients']==0]

There are 38 columns with a coefficient of zero.
These columns are:


,Features,Coefficients
39,Laptop friendly workspace,0.0
78,Carbon monoxide detector,0.0
97,Smoking allowed,0.0
141,Smoke detector,0.0
249,nw7,0.0
263,se1p,0.0
268,se23,-0.0
353,1bd flat,0.0
378,apart,0.0
427,bed garden,-0.0


In [20]:
# view the bottom 25 values:
(coef_cv.sort_values(by='Coefficients', ascending=False)).tail(25)

,Features,Coefficients
308,sw95,-4408.539654
284,sw13,-4480.864851
607,family house,-4525.920079
606,family home,-4583.586492
706,house london,-4605.066185
825,near central,-4798.937254
130,Bathroom essentials,-4998.896817
929,room en,-5251.832149
1043,triple room,-5256.561700
740,lift,-5338.143865


There is an interesting mix of values within the top and bottom coefficient groups, as well as those that have zero coefficients. Unsurprisingly, Average Daily Rate (Native) has a large positive coefficient, as do many central london postcodes. Interestingly, 'covent' has the third largest positive coefficient, but 'covent garden' has the lowest. 

The next model, will use the TF-IDF processed text data, which only included bigrams. This should provide a clearer, more easy to interpret result.

### M1 - V2: Lasso TF-IDF

The same process will no be used on the text that has been processed using the Tf-IDF method:

In [21]:
# split independent and target data between train and test sets:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf_scaled, y, test_size = 0.25, random_state=42)

In [22]:
# 1. Instantiate the model
alpha = 0.1
max_iter = 1000
lasso_tfidf = Lasso(alpha = alpha, max_iter = max_iter)

# 2. Fit the model
lasso_tfidf.fit(X_train_tfidf, y_train_tfidf)

# 3. Determine the model coefficients
lasso_coef = lasso_tfidf.coef_
feature_names = X_train_tfidf.columns

coef_tfidf = pd.DataFrame({'Features': feature_names, 'Coefficients': lasso_coef})

In [23]:
# Make predictions
y_pred_train_tfidf = lasso_tfidf.predict(X_train_tfidf)
y_pred_tfidf = lasso_tfidf.predict(X_test_tfidf)



# Evaluate model
print(f'Train MAE --- {round(mean_absolute_error(y_train_tfidf, y_pred_train_tfidf),1)}')
print(f'Test MAE ---- {round(mean_absolute_error(y_test_tfidf, y_pred_tfidf),1)}')
print(f'Train R2 ---- {round(r2_score(y_train_tfidf, y_pred_train_tfidf),3)}')
print(f'Test R2 ----- {round(r2_score(y_test_tfidf, y_pred_tfidf),3)}')

Train MAE --- 6597.2
Test MAE ---- 6654.0
Train R2 ---- 0.718
Test R2 ----- 0.72


This model has scored almost identically to the previous lasso model (using the count vectorized column), the test MAE is sligtly lower on this second model, indicatin the model is performing slightly better on predicting the test data.

The coefficients will be evaluated for the different features:

In [24]:
# determine the top 20 feature coefficients;
(coef_tfidf.sort_values(by='Coefficients', ascending=False)).head(25)

,Features,Coefficients
342,wc2a,36247.372152
202,ec3a,22851.063304
85,Stair gates,20937.224013
291,sw1a,16255.714266
199,ec2r,13253.278727
347,wc2r,12259.159910
198,ec2n,12202.942966
340,wc1v,10523.858632
118,Self check-in,9267.972484
20,Average Daily Rate (Native),8966.203428


In [25]:
# view columns will a coefficient of 0:
print(f"There are {len(coef_tfidf[coef_tfidf['Coefficients']==0])} columns with a coefficient of zero.")

print("These columns are:")
coef_tfidf[coef_tfidf['Coefficients']==0]

There are 17 columns with a coefficient of zero.
These columns are:


,Features,Coefficients
39,Laptop friendly workspace,0.0
78,Carbon monoxide detector,0.0
97,Smoking allowed,0.0
116,Full kitchen,-0.0
141,Smoke detector,0.0
209,ec4r,0.0
243,nw1w,0.0
263,se1p,0.0
268,se23,-0.0
350,1bd flat,-0.0


In [26]:
# view the 25 features with the lowest coefficients:
(coef_tfidf.sort_values(by='Coefficients', ascending=False)).tail(25)

,Features,Coefficients
133,Table corner guards,-3608.536977
219,n11,-3693.356895
297,sw1x,-3912.037939
379,bed bath,-3970.700796
150,Beach essentials,-4022.675741
533,luxury bed,-4074.485106
423,close tube,-4092.459151
353,2bed 2bath,-4243.048549
453,family house,-4244.461086
383,bed home,-4480.859749


For this second model using the TF-IDF embedded dataframe, there are quite a few feature that appear in the both top and bottom 20 for the two lasso models, mainly columns that are not components of the Listing title column. 

A more advanced Lasso model will be created using the TF-IDF data, this model will use a GridSearchCV, to find the optimial hyper parameters, mainly being the value for alpha. The same train test split datasets will be used.

In [27]:
# define model type:
lasso = Lasso()

# define parameters:
param_grid = {'alpha': [1, 2, 5, 10],
            'max_iter' : [1000, 5000]}

# define the grid search:
Lasso_gs = GridSearchCV(estimator=lasso, param_grid=param_grid, cv=5, n_jobs=-1, verbose=5)

# grid search
Lasso_gs.fit(X_train_tfidf, y_train_tfidf)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


exception calling callback for <Future at 0x1fce62eb790 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\blcm9\anaconda3\envs\capstone\lib\site-packages\joblib\externals\loky\process_executor.py", line 391, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "C:\Users\blcm9\anaconda3\envs\capstone\lib\multiprocessing\queues.py", line 111, in get
    res = self._recv_bytes()
  File "C:\Users\blcm9\anaconda3\envs\capstone\lib\multiprocessing\connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "C:\Users\blcm9\anaconda3\envs\capstone\lib\multiprocessing\connection.py", line 318, in _recv_bytes
    return self._get_more_data(ov, maxsize)
  File "C:\Users\blcm9\anaconda3\envs\capstone\lib\multiprocessing\connection.py", line 340, in _get_more_data
    ov, err = _winapi.ReadFile(self._handle, left, overlapped=True)
MemoryE

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
# determine the best parameters used:
Lasso_gs.best_params_

In [ ]:
# Make predictions:
y_pred_train_lasso = Lasso_gs.predict(X_train_tfidf)
y_pred_lasso = Lasso_gs.predict(X_test_tfidf)

# Evaluating the model
print(f'Train MAE --- {mean_absolute_error(y_train_tfidf, y_pred_train_lasso)}')
print(f'Test MAE ---- {mean_absolute_error(y_test_tfidf, y_pred_lasso)}')
print(f'Train R2 ---- {r2_score(y_train_tfidf, y_pred_train_lasso)}')
print(f'Test R2 ----- {r2_score(y_test_tfidf, y_pred_lasso)}')

This model using the GridSearchCV to determine the best parameters for the Lasso model, has performed the best of the three completed lasso models. With the most notable difference beign the Test MAE value being reduced - it is now slightly lower than the train MAE, showing the model has trained on the dataset very well and if effective at predicting the target column.

The feature coefficients will now be viewed. The best model parameters will be saved and fitted to the training dataset again, to allow for the coefficients to be viewed:

In [ ]:
# save the best parameters found from the gridsearch:
best_params = Lasso_gs.best_params_

# create a model with these parameters:
best_model = Lasso(**best_params)

# fit this model to the train dataframes:
best_model.fit(X_train_tfidf, y_train_tfidf)

In [ ]:
# determine the coefficients for the features determined by this model:
lasso_coef = best_model.coef_
feature_names = X_train_tfidf.columns

coef_tfidf = pd.DataFrame({'Features': feature_names, 'Coefficients': lasso_coef})

In [ ]:
# view the features with the top 25 coefficient values:
coef_tfidf.sort_values(by='Coefficients', ascending=False).head(25)

In [ ]:
top_10 = coef_tfidf.sort_values(by='Coefficients', ascending=False).head(10)

# visualise the top 10 most positive coefficients:
plt.figure()
plt.barh(top_10['Features'], top_10['Coefficients'])
plt.title('Model 1: Lasso Coefficient - Top 10 Features')
plt.xlabel('Coefficient')
plt.gca().invert_yaxis()

plt.show()

In [ ]:
# view columns will a coefficient of 0:
print(f"There are {len(coef_tfidf[coef_tfidf['Coefficients']==0])} columns with a coefficient of zero.")

print("These columns are:")
coef_tfidf[coef_tfidf['Coefficients']==0]

In [ ]:
# view the 25 features with the lowest coefficients:
(coef_tfidf.sort_values(by='Coefficients', ascending=False)).tail(25)

In [ ]:
bottom_10 = coef_tfidf.sort_values(by='Coefficients', ascending=False).tail(10)

# visualise the top 10 most positive coefficients:
plt.figure()
plt.barh(bottom_10['Features'], bottom_10['Coefficients'])
plt.title('Lasso Coefficient - Top 10 Features')
plt.xlabel('Coefficient')
plt.gca().invert_yaxis()

plt.show()

There are many features which occur again in the top and bottom 25 coefficients for features in the GridSearchV lasso model result. The main difference is the quantity of features with zero coefficient. The optimising of the parameters has produced a model that has deemed 400 of the features to have no significant influence on predicting the target column. 

### M1 - V3: Lasso Model -  Dropped Semantically Obvious Columns

A final Lasso model will be complete, in this model some of the previously determined high coefficient columns will be dropped. There are several columns, such as 'Count Reservation Days LTM', 'Count Available Days LTM' where the high postive coefficient is completely expected and hence their inclusion isn't particularly insightful (as previously mentioned, it is obvious that charge more per night and having a highly occupied property will lead to a larger annual revenue). These columns also have high multi-collinearity, which can distort the model.

In [ ]:
X_train_2 = X_train_tfidf.drop(columns = ['Count Available Days LTM', 'Count Blocked Days LTM', 'Count Reservation Days LTM', 'Number of Bookings LTM'])
X_test_2 = X_test_tfidf.drop(columns = ['Count Available Days LTM', 'Count Blocked Days LTM', 'Count Reservation Days LTM', 'Number of Bookings LTM'])

In [ ]:
# create new gridsearchcv model with the reduced dataframes:
lasso = Lasso()
# define parameters:
param_grid = {'alpha': [1, 2, 5, 10],
            'max_iter' : [1000, 5000]}

# define the grid search:
Lasso_gs_2 = GridSearchCV(estimator=lasso, param_grid=param_grid, cv=5, n_jobs=-1, verbose=5)

# grid search
Lasso_gs_2.fit(X_train_2, y_train_tfidf)

In [ ]:
# Make predictions:
y_pred_train_lasso = Lasso_gs_2.predict(X_train_2)
y_pred_lasso = Lasso_gs_2.predict(X_test_2)

# Evaluating the model
print(f'Train MAE --- {mean_absolute_error(y_train_tfidf, y_pred_train_lasso)}')
print(f'Test MAE ---- {mean_absolute_error(y_test_tfidf, y_pred_lasso)}')
print(f'Train R2 ---- {r2_score(y_train_tfidf, y_pred_train_lasso)}')
print(f'Test R2 ----- {r2_score(y_test_tfidf, y_pred_lasso)}')

By removing many of the columns which are semantically obviously influencial to the annual revenue, the model has predictably performed worse. The MAE has increased for both the train and target values (although both are very similar) and the accuracy has reduced. A accuracy score of 0.49 for the test and 0.46 is still substantial, so the model is still effective in predicting some degree of the variances within the target column after the removal of several key columns.

The feature coefficients will be viewed:

In [ ]:
# save the best parameters found from the gridsearch:
best_params = Lasso_gs_2.best_params_

# create a model with these parameters:
best_model = Lasso(**best_params)

# fit this model to the train dataframes:
best_model.fit(X_train_2, y_train_tfidf)

In [ ]:
# determine the coefficients for the features determined by this model:
lasso_coef = best_model.coef_
feature_names = X_train_2.columns

coef_tfidf = pd.DataFrame({'Features': feature_names, 'Coefficients': lasso_coef})

In [ ]:
# view the features with the top 25 coefficient values:
coef_tfidf.sort_values(by='Coefficients', ascending=False).head(25)

In [ ]:
top_10 = coef_tfidf.sort_values(by='Coefficients', ascending=False).head(10)

# visualise the top 10 most positive coefficients:
plt.figure()
plt.barh(top_10['Features'], top_10['Coefficients'])
plt.title('Lasso Coefficient - Top 10 Features')
plt.xlabel('Coefficient')
plt.gca().invert_yaxis()

plt.show()

In [ ]:
# view columns will a coefficient of 0:
print(f"There are {len(coef_tfidf[coef_tfidf['Coefficients']==0])} columns with a coefficient of zero.")

print("These columns are:")
coef_tfidf[coef_tfidf['Coefficients']==0]

In [ ]:
# view the 25 features with the lowest coefficients:
(coef_tfidf.sort_values(by='Coefficients', ascending=False)).tail(25)

In [ ]:
bottom_10 = coef_tfidf.sort_values(by='Coefficients', ascending=False).tail(10)

# visualise the top 10 most positive coefficients:
plt.figure()
plt.barh(bottom_10['Features'], bottom_10['Coefficients'])
plt.title('Lasso Coefficient - Top 10 Features')
plt.xlabel('Coefficient')
plt.gca().invert_yaxis()

plt.show()

By dropping the mentioned columns, the features appearing in the top and bottom 10 coefficients has remained relatively constant. Although, the coefficient for the 'Average Daily Rate (Native)' how now increased slightly, meaning the model now implies the 'Average Daily Rate (Native)' now have more of an impact on influencing the target column than it did previously.

### M1 - V4: Lasso - Dropped 'Average Daily Rate'

The 'Average Daily Rate' column has been included in all previous lasso models. As discovered, the coefficient for this column is the largest positive value. Another lasso model will be run, with this column being dropped. 

Knowing the 'Average Daily Rate' of a property is influencial in whether the property makes a higher annual revenue isn't a particularly interesting discovery. It is good information to know, as it demonstrate the model is working as expected, but completing a model with its ommision next will be beneficial in identifying other, less obvious factors.

In [ ]:
# dropping the average daily rate column:
X_train_3 = X_train_2.drop(columns='Average Daily Rate (Native)')
X_test_3= X_test_2.drop(columns='Average Daily Rate (Native)')

In [ ]:
lasso = Lasso()
# define parameters:
param_grid = {'alpha': [1, 2, 4, 6],
            'max_iter' : [1000, 5000]}

# define the grid search:
Lasso_gs_3 = GridSearchCV(estimator=lasso, param_grid=param_grid, cv=5, n_jobs=-1, verbose=5)

# grid search
Lasso_gs_3.fit(X_train_3, y_train_tfidf)

In [ ]:
# Make predictions:
y_pred_train_lasso = Lasso_gs_3.predict(X_train_3)
y_pred_lasso = Lasso_gs_3.predict(X_test_3)

# Evaluating the model
print(f'Train MAE --- {round(mean_absolute_error(y_train_tfidf, y_pred_train_lasso),2)}')
print(f'Test MAE ---- {round(mean_absolute_error(y_test_tfidf, y_pred_lasso),2)}')
print(f'Train R2 ---- {round(r2_score(y_train_tfidf, y_pred_train_lasso),2)}')
print(f'Test R2 ----- {round(r2_score(y_test_tfidf, y_pred_lasso),2)}')

The removal of the 'Average Daily Rate' column has again caused the model to perform worse. The train and test MAE have both increased and both their accuracies have decreased. Although, the scores for MAE and r2 are very similar for the train and test results, hence the model is not overfitting.

A accuracy of approximately 0.4 for both the train and test dataframes is significantly lower than the original lasso model result. But the removal of certain semantically obvious columns, means that the results and specific columns coefficient values, are now more insightful and lend towards more useful features being identified in determining which property characteristics can lead to an increase/decrease in annual revenue.

The coefficients of the features will now be determined:

In [ ]:
# save the best parameters found from the gridsearch:
best_params = Lasso_gs_3.best_params_

# create a model with these parameters:
best_model = Lasso(**best_params)

# fit this model to the train dataframes:
best_model.fit(X_train_3, y_train_tfidf)

In [ ]:
# determine the coefficients for the features determined by this model:
lasso_coef = best_model.coef_
feature_names = X_train_3.columns

coef_tfidf = pd.DataFrame({'Features': feature_names, 'Coefficients': lasso_coef})

In [ ]:
# view the features with the top 25 coefficient values:
coef_tfidf.sort_values(by='Coefficients', ascending=False).head(25)

In [ ]:
top_10 = coef_tfidf.sort_values(by='Coefficients', ascending=False).head(10)

# visualise the top 10 most positive coefficients:
plt.figure()
plt.barh(top_10['Features'], top_10['Coefficients'])
plt.title('Lasso Coefficient - Top 10 Features')
plt.xlabel('Coefficient')
plt.gca().invert_yaxis()

plt.show()

In [ ]:
# view columns will a coefficient of 0:
print(f"There are {len(coef_tfidf[coef_tfidf['Coefficients']==0])} columns with a coefficient of zero.")

print("These columns are:")
coef_tfidf[coef_tfidf['Coefficients']==0]

In [ ]:
# view the 25 features with the lowest coefficients:
(coef_tfidf.sort_values(by='Coefficients', ascending=False)).tail(25)

In [ ]:
bottom_10 = coef_tfidf.sort_values(by='Coefficients', ascending=False).tail(10)

# visualise the top 10 most positive coefficients:
plt.figure()
plt.barh(bottom_10['Features'], bottom_10['Coefficients'])
plt.title('Lasso Coefficient - Top 10 Features')
plt.xlabel('Coefficient')
plt.gca().invert_yaxis()

plt.show()

Dropping the 'Average Daily Rate (Native)' column, has been influencial in adjusting the other features coefficient values. The top 10 coefficient values are now dominated by various postcode values, with all postcode regions being central London locations. Also, the quantity of features with coefficient values of zero has also been reduced, meaning more coefficients are now considered influencial in predicting the target column.

This has been an effective implementation of the lasso model on the various features. The models produced have varied in accuracy and MAE, but the final model, with severly trimmed down features has still performed relatively well and thus, insights can be gauged from its output.

## Model 2: XGBoost Regressor
### V1: Entire dataset

A XGBoost model will now be used for the second modelling approach. An XGBoost model has been selected due to the large quantity of binary values within the dataframe; meaning this type of modelling is more effective than random forests/standard decision tree modelling.

The same dataframe, with the the TF-IDF 'Listing title' embedded values and the standard scaled continuous variables will be used for this modelling. Hence, the dataframes from the original TF-IDF dataframes split can be used.

**I have had issues getting the model to run due to memory capacity. I am therefore completing another train text split, to essential act as creating a data subset, which will be more manageable for my device to process**

In [ ]:
# complete second train test split for processing purposes:
X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X_train_tfidf, y_train_tfidf, test_size = 0.25, random_state=42)

In [ ]:
# required import:
from xgboost import XGBRegressor

# To set up a temporary directory for caching pipeline results
from tempfile import mkdtemp

# To build a pipeline
from sklearn.pipeline import Pipeline

In [ ]:
# define regressor
xgb = XGBRegressor()

# define param grid:
params = {
    'n_estimators' : [100, 150],
    'max_depth' : [8, 10],
    'learning_rate' : [0.2, 0.4]
}

# create gridsearch to determine the best parameters:
xgb_gs = GridSearchCV(estimator = xgb, param_grid=params, cv=5, n_jobs=-1, verbose=5)

In [ ]:
# fit to the necessary training data:
xgb_gs.fit(X_train_sub, y_train_sub)

In [ ]:
# determine the model best parameters:
best_params = xgb_gs.best_params_
best_params

In [ ]:
# create new model with these parameters:
best_xgb = XGBRegressor(**best_params)

# refit the model onto the train dataset, with the best parameters:
best_xgb.fit(X_train_sub, y_train_sub)

In [ ]:
# Make predictions
y_pred_train_xg = best_xgb.predict(X_train_sub)
y_pred_xg = best_xgb.predict(X_test_sub)



# Evaluate model
print(f'Train MAE --- {round(mean_absolute_error(y_train_sub, y_pred_train_xg),2)}')
print(f'Test MAE ---- {round(mean_absolute_error(y_test_sub, y_pred_xg),2)}')
print(f'Train R2 ---- {round(r2_score(y_train_sub, y_pred_train_xg), 4)}')
print(f'Test R2 ----- {round(r2_score(y_test_sub, y_pred_xg),4)}')

Both the train and test datasets have scored very well for this first XGBoost Regressor model. The accuracy is very high for both the train and test datasets. However, the test MAE is significantly higher than the train MAE, suggesting that some degree of overfitting has occured. Hence, another model will be completed altering certain properties and attempting to reduce the amount of overfitting experienced.

### M2 - V2: Dropping Semanitcally Obvious Columns
This second version of the XGBoost model will be completed using the same training dataset that was used for the final Lasso model. This is a dataset which has several multicollinear independent variable removed.

The same GridSearchCV and parameters will be used for this model:

In [ ]:
X_train_sub2 = X_train_sub.drop(columns = ['Count Available Days LTM', 'Count Blocked Days LTM', 'Count Reservation Days LTM', 'Number of Bookings LTM'])
X_test_sub2 = X_test_sub.drop(columns = ['Count Available Days LTM', 'Count Blocked Days LTM', 'Count Reservation Days LTM', 'Number of Bookings LTM'])

In [ ]:
# define regressor
xgb = XGBRegressor()

# define param grid:
params = {
 'n_estimators' : [75, 100],
    'max_depth' : [3, 4],
    'learning_rate' : [0.01, 0.05],
    'min_child_weight' : [8, 10]
}

# create gridsearch to determine the best parameters:
xgb_gs_2 = GridSearchCV(estimator = xgb, param_grid=params, cv=5, n_jobs=-1, verbose=5)

In [ ]:
xgb_gs_2.fit(X_train_sub2, y_train_sub)

In [ ]:
# determine the model best parameters:
best_params = xgb_gs_2.best_params_
best_params

In [ ]:
# create new model with these parameters:
best_xgb_2 = XGBRegressor(**best_params)

# refit the model onto the train dataset, with the best parameters:
best_xgb_2.fit(X_train_sub2, y_train_sub)

In [ ]:
# Make predictions
y_pred_train_xg2 = best_xgb_2.predict(X_train_sub2)
y_pred_xg2 = best_xgb_2.predict(X_test_sub2)



# Evaluate model
print(f'Train MAE --- {round(mean_absolute_error(y_train_sub, y_pred_train_xg2),2)}')
print(f'Test MAE ---- {round(mean_absolute_error(y_test_sub, y_pred_xg2),2)}')
print(f'Train R2 ---- {round(r2_score(y_train_sub, y_pred_train_xg2), 4)}')
print(f'Test R2 ----- {round(r2_score(y_test_sub, y_pred_xg2),4)}')

The model with the various dropped column is running fairly well. The parameters have been tuned within the GridSearchCV to prevent overfitting. The model accuracy have reduced significantly from the previous model, but this is not surprising given the influencial columns that have been dropped. The MAE and r2 values are fairly similar for their train and test scores, the training accuracy score is about 0.05 larger than the test score, suggesting some overfitting has occured, although the degree of which is not too severe and hence the model is valid.

### M2: V3 - Dropping the 'Average Daily Rate' Column

The next version of the model will involve dropping the 'Average Daily Rate' column. As this column is a very important column semantically, it will be informative to see the impact dropping it has on the models results:

In [ ]:
# create new dataframes with the 'Average Daily Rate' column being dropped:
X_train_sub3 = X_train_sub2.drop(columns = ['Average Daily Rate (Native)'])
X_test_sub3 = X_test_sub2.drop(columns = ['Average Daily Rate (Native)'])

In [ ]:
# define regressor
xgb = XGBRegressor()

# define param grid:
params = {
    'n_estimators' : [75, 100],
    'max_depth' : [3, 4],
    'learning_rate' : [0.01, 0.05],
    'min_child_weight' : [8, 10]
}

# create gridsearch to determine the best parameters:
xgb_gs_3 = GridSearchCV(estimator = xgb, param_grid=params, cv=5, n_jobs=-1, verbose=5)

In [ ]:
# fit to dataframe
xgb_gs_3.fit(X_train_sub3, y_train_sub)

In [ ]:
# determine the model best parameters:
best_params = xgb_gs_3.best_params_
best_params

In [ ]:
# create new model with these parameters:
best_xgb_3 = XGBRegressor(**best_params)

# refit the model onto the train dataset, with the best parameters:
best_xgb_3.fit(X_train_sub3, y_train_sub)

In [ ]:
# Make predictions
y_pred_train_xg3 = best_xgb_3.predict(X_train_sub3)
y_pred_xg3 = best_xgb_3.predict(X_test_sub3)



# Evaluate model
print(f'Train MAE --- {round(mean_absolute_error(y_train_sub, y_pred_train_xg3),2)}')
print(f'Test MAE ---- {round(mean_absolute_error(y_test_sub, y_pred_xg3),2)}')
print(f'Train R2 ---- {round(r2_score(y_train_sub, y_pred_train_xg3), 4)}')
print(f'Test R2 ----- {round(r2_score(y_test_sub, y_pred_xg3),4)}')

This final model is performing relatively well and the MAE scores and accuracy scores are fairly similar. The most is unsurprisingly performing worse than the previous iteration, now that the 'Average Daily Rate' column has been dropped.

Although, considering the majority of columns that have been deemed too semantically obvious to be included, have been dropped. An accuracy of approx 0.5 is a good result. In that the remaining columns are predicting 50% of the variance within the target column.

The XGBRegressor models will be evaluated further.

To begin, the models feature importances will be viewed. These feature importance values will be viewed for the model containing the 'Average Daily Rate' and the model where this column has been dropped (M2:V2 and M2:V3).

### Model Evaluation: M2:V2:

In [ ]:
# Get the feature importances
feature_importances = best_xgb_2.feature_importances_

# create a dataframe to store the feature importances with corresponding feature names:
importance_df = pd.DataFrame({'Features' : X_train_sub2.columns, 'Importance' : feature_importances})

importance_xg1 = importance_df.sort_values(by = 'Importance', ascending=False).head(10)
importance_xg1

In [ ]:
plt.figure()
plt.barh(importance_xg1['Features'], importance_xg1['Importance'])
plt.title('Model 1: Importance Value - Top 10 Features')
plt.xlabel('Feature Importance')
plt.gca().invert_yaxis()

plt.show()

In [ ]:
# required import
import shap

shap.initjs()

# create an explainer object
explainer = shap.Explainer(best_xgb_2)

# compue SHAP values
shap_values = explainer.shap_values(X_train_sub2)

shap.summary_plot(shap_values, features=X_train_sub2, feature_names=X_train_sub2.columns)

The output for the SHAP values for the XGBRegressor results are displayed above. As expected, the 'Average Daily Rate' column has the most significant impact on the model. It's impact is that much greater than the other columns, that it drawfs the other columns visualisations. 

Hence, another SHAP chart will be created for the XGBRegressor model which had this column dropped. The results from this second graph, should provide some more useful insights regarding columns impact within the model.

### Evaluation for M2:V3

In [ ]:
# Get the feature importances
feature_importances2 = best_xgb_3.feature_importances_

# create a dataframe to store the feature importances with corresponding feature names:
importance_df2 = pd.DataFrame({'Features' : X_train_sub3.columns, 'Importance' : feature_importances2})

importance_2 = importance_df2.sort_values(by = 'Importance', ascending=False).head(10)
importance_2

In [ ]:
plt.figure()
plt.barh(importance_2['Features'], importance_2['Importance'])
plt.title('Model 2: Importance Value - Top 10 Features')
plt.xlabel('Feature Importance')
plt.gca().invert_yaxis()

plt.show()

In [ ]:
shap.initjs()

# create an explainer object
explainer = shap.Explainer(best_xgb_3)

# compue SHAP values
shap_values = explainer.shap_values(X_train_sub3)

shap.summary_plot(shap_values, features=X_train_sub3, feature_names=X_train_sub3.columns)

The importance factor and SHAP values for the final XGBRegressor models, provide some more interesting insights. There are some column occurances which are not surprising, in that 'Bedroom', 'bathrooms', 'Airbnb superhost', 'Air conditioning' are all important features which can impact the model output. Some good points that are definitely aspects property owners can gauge some additional meaning from are features such as 'coffe maker', 'free street parking', dishwasher', these are all features that can influence how a prospective property buyer would seek our their purchase, or some components that existing property owners can incorporate into their Airbnb asset. 

Further investigation of the individual features impact on the model can be assessed in following analysis and model iterations.

# Conclusion

To conclude this advanced modelling section. Numerous iterations of the Lasso model and the XGBoost model have been undertaken. Through completing these models, it can be confirmed that there are many unique property features (beyond the obvious ones), which are influencial in determining the annual revenue of a Airbnb property. The models have performed well when fitting to the training data and then at predicting the test data.

Through various independent variable tweaking and hyperparamter tuning, good scores have been achieved across all models. The models have performed in a manner tha confirms the initial hypothesis for this project, and have resulted in the ability of individual property feature value to be assessed quantitatively. 

Future iterations of this model can be undertaken to explore feature impacts to a greater extent. As well as, the same principles and workflow being implemented to Airbnb properties for all cities across the globe.